In [ ]:
CREATE SCHEMA Authentication

In [ ]:
CREATE TABLE Authentication.Users(
    UserID int IDENTITY(1,1) PRIMARY KEY,
    Username NVARCHAR(20),
    Password NVARCHAR(256),
    Image NVARCHAR(max),
    UserType int
)
CREATE TABLE Authentication.DeletedUsers(
    UserID int PRIMARY KEY,
    Username NVARCHAR(20),
    Password NVARCHAR(256),
    Image NVARCHAR(max),
    UserType int
)

In [ ]:
CREATE PROCEDURE Authentication.UserSignUp
    @Username NVARCHAR(255),
    @Password NVARCHAR(255),
    @Image NVARCHAR(MAX)
AS
BEGIN

    -- Check if the username already exists in Users or DeletedUsers
    IF NOT EXISTS (SELECT 1 FROM Users WHERE Username = @Username)
        AND NOT EXISTS (SELECT 1 FROM DeletedUsers WHERE Username = @Username)
    BEGIN
        BEGIN TRY
            BEGIN TRANSACTION;

            -- Insert the user into the Users table
            INSERT INTO Users (Username, Password, Image, UserType) VALUES (@Username, @Password, @Image, 0);
            COMMIT TRANSACTION;
        END TRY
        BEGIN CATCH
            IF @@TRANCOUNT > 0
                ROLLBACK TRANSACTION;

            -- Handle errors as needed
            THROW;
        END CATCH;
    END
    ELSE
    BEGIN
        -- Username already exists, handle the situation accordingly
        RAISERROR('Username already exists', 16, 1);
    END
END;


SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE PROCEDURE [Authentication].[InsertKeystrokesIntoDynamicTable]
    @TableName NVARCHAR(100),
    @Keystrokes NVARCHAR(max),
    @Expected int
AS
BEGIN
    DECLARE @sql NVARCHAR(MAX)

    -- Construct the dynamic SQL statement
    SET @sql = 'INSERT INTO [Authentication].' + QUOTENAME(@TableName) + ' (Keystrokes, Expected) VALUES (@Keystrokes, 0)'

    -- Execute the dynamic SQL
    EXEC sp_executesql @sql, N'@Keystrokes NVARCHAR(max), @Expected int', @Keystrokes, @Expected
END
GO


SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE PROCEDURE [Authentication].[GetKeystrokes]
    @TableName NVARCHAR(max)  -- Assuming the table name is passed as a parameter
AS
BEGIN
    -- Declare variables for dynamic SQL
    DECLARE @SQL NVARCHAR(MAX);

    -- Construct the dynamic SQL query
    SET @SQL = N'SELECT Keystrokes, Expected FROM Authentication.' + QUOTENAME(@TableName);

    -- Execute the dynamic SQL query
    EXEC sp_executesql @SQL;
END;
GO


SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE PROCEDURE [Authentication].[DeleteUser]
    @Username NVARCHAR(max),
    @UserID int
AS
BEGIN

    DECLARE @OldTableName NVARCHAR(255);
    DECLARE @NewTableName NVARCHAR(255);

    -- Get the current table name associated with the username
    

    -- Generate a new table name (e.g., by appending a timestamp)
    SET @NewTableName = 'DeletedUser' + @Username;
    SET @OldTableName = 'Authentication.' + @Username;
    BEGIN TRY
        BEGIN TRANSACTION;

        -- Delete the user from the 'users' table
        DELETE FROM Users WHERE UserID = @UserID;

        -- Rename the user's keystroke table
        EXEC sp_rename @OldTableName, @NewTableName;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        IF @@TRANCOUNT > 0
            ROLLBACK TRANSACTION;

        -- Handle errors as needed
        THROW;
    END CATCH;
END;
GO

SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

CREATE PROCEDURE [Authentication].[SelectSpecificUserData]
    @Username NVARCHAR(50)
AS
BEGIN
    SELECT * FROM Authentication.Users WHERE Username = @Username
END;
GO



CREATE PROCEDURE Authentication.AdminChange
    @UserID INT,
    @UserType int
AS
BEGIN
    SET NOCOUNT ON;

    -- Update the value for the selected row
    UPDATE Authentication.Users SET UserType = @UserType WHERE UserID = @UserID;   

    -- Check if any rows were affected
    IF @@ROWCOUNT > 0
    BEGIN
        -- Rows were affected, operation successful
        SELECT 'Update successful.' AS Result;
    END
    ELSE
    BEGIN
        -- No rows were affected, user not found
        SELECT 'User not found.' AS Result;
    END
END


SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE PROCEDURE [Authentication].[RecoverUser]
    @Username NVARCHAR(max),
    @UserID int
AS
BEGIN

    DECLARE @OldTableName NVARCHAR(255);
    DECLARE @NewTableName NVARCHAR(255);

    -- Get the current table name associated with the username
    

    -- Generate a new table name (e.g., by appending a timestamp)
    SET @NewTableName = @Username;
    SET @OldTableName = 'Authentication.DeletedUser' + @Username;
    BEGIN TRY
        BEGIN TRANSACTION;

        -- Delete the user from the 'users' table
        DELETE FROM DeletedUsers WHERE UserID = @UserID;

        -- Rename the user's keystroke table
        EXEC sp_rename @OldTableName, @NewTableName;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        IF @@TRANCOUNT > 0
            ROLLBACK TRANSACTION;

        -- Handle errors as needed
        THROW;
    END CATCH;
END;
GO

CREATE PROCEDURE Authentication.ChangePassword(
    @UserID int,
    @Password nvarchar(max),
    @TableName nvarchar(max)
)
AS
BEGIN

    -- Update the value for the selected row
    UPDATE Authentication.Users SET Password = @Password WHERE UserID = @UserID;
    DECLARE @SqlStatement NVARCHAR(MAX);
    SET @SqlStatement = 'DELETE FROM Authentication.' + QUOTENAME(@TableName) + 'WHERE 1=1'
    EXEC sp_executesql @SqlStatement;

    -- Check if any rows were affected
    IF @@ROWCOUNT > 0
    BEGIN
        -- Rows were affected, operation successful
        SELECT 'Update successful.' AS Result;
    END
    ELSE
    BEGIN
        -- No rows were affected, user not found
        SELECT 'User not found.' AS Result;
    END
END

In [ ]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TRIGGER [Authentication].[CreateUserTable]
ON [Authentication].[Users]
AFTER INSERT
AS
BEGIN
    IF (UPDATE(Username))
    BEGIN
        DECLARE @Username VARCHAR(MAX);
        SELECT @Username = Username FROM INSERTED;

        DECLARE @TableName NVARCHAR(255);
        SET @TableName = QUOTENAME(@Username); -- Ensuring table name is properly quoted

        DECLARE @SQL NVARCHAR(MAX);
        SET @SQL = N'
            IF NOT EXISTS (SELECT 1 FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = ''' + @TableName + ''')
            BEGIN
                CREATE TABLE Authentication.' + @TableName + ' (
                    PatternNumber INT PRIMARY KEY IDENTITY(1,1),
                    Keystrokes VARCHAR(MAX),
                    Expected int
                );
            END';
        
        EXEC sp_executesql @SQL;
    END
END
GO
ALTER TABLE [Authentication].[Users] ENABLE TRIGGER [CreateUserTable]
GO

SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TRIGGER [Authentication].[UserDeleted]
ON [Authentication].[Users]
FOR DELETE
AS
BEGIN

    -- Insert deleted rows into the DeletedUsers table
    INSERT INTO Authentication.DeletedUsers (UserID, Username, Password, Image, UserType)
    SELECT UserID, Username, Password, Image, UserType
    FROM DELETED;
END;
GO
ALTER TABLE [Authentication].[Users] ENABLE TRIGGER [UserDeleted]
GO

SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TRIGGER [Authentication].[UserRecovered]
ON [Authentication].[DeletedUsers]
FOR DELETE
AS
BEGIN

    -- Insert deleted rows into the DeletedUsers table
    SET IDENTITY_INSERT Authentication.Users ON;
    INSERT INTO Authentication.Users (UserID, Username, Password, Image, UserType)
    SELECT UserID, Username, Password, Image, UserType
    FROM DELETED;
    SET IDENTITY_INSERT Authentication.Users OFF;
END;
GO
ALTER TABLE [Authentication].[DeletedUsers] ENABLE TRIGGER [UserRecovered]
GO



In [ ]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE VIEW [Authentication].[AllUsersView] AS SELECT [UserID], [Username], [Image], [UserType] FROM [Authentication].[Users]
GO

SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE VIEW [Authentication].[AllDeletedUsersView] AS SELECT [UserID], [Username], [Image], [UserType] FROM [Authentication].[DeletedUsers]
GO
